In [ ]:
# -*- coding: utf-8 -*-

import json

import pandas as pd

import plotly
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html

# Import data 
Towncensus_data = pd.read_csv('TownCensus_maharashtra_1_00.csv')

with open('TownCensus_maharashtra_1_00.json') as json_data:
    Maharashtra_data = json.load(json_data)

# mapbox token
mapbox_accesstoken = 'pk.eyJ1Ijoid2luZG1hbjAwNTEwcCIsImEiOiJja3Fhd280b3EwMTQ0MnZwNzUzZWt4MnVlIn0.AKpTtyDrEI2aIzJXBRHU1Q'

# This is the part to create plotly fig
########################################################
town_name=Towncensus_data['town_name'].str.title().tolist()

pl_deep=[[0.0, 'rgb(253, 253, 204)'],
         [0.1, 'rgb(201, 235, 177)'],
         [0.2, 'rgb(145, 216, 163)'],
         [0.3, 'rgb(102, 194, 163)'],
         [0.4, 'rgb(81, 168, 162)'],
         [0.5, 'rgb(72, 141, 157)'],
         [0.6, 'rgb(64, 117, 152)'],
         [0.7, 'rgb(61, 90, 146)'],
         [0.8, 'rgb(65, 64, 123)'],
         [0.9, 'rgb(55, 44, 80)'],
         [1.0, 'rgb(39, 26, 44)']]

Types = ['maternity_','maternit_1','maternit_2','mat_child_']    

trace1 = []    
    
# town_name order should be the same as "id" passed to location
for q in Types:
    trace1.append(go.Choroplethmapbox(
        geojson = Maharashtra_data,
        locations = Towncensus_data['id'].tolist(),
        z = Towncensus_data[q].tolist(), 
        colorscale = pl_deep,
        text = town_name, 
        colorbar = dict(thickness=20, ticklen=3),
        marker_line_width=0, marker_opacity=0.7,
        visible=False,
        subplot='mapbox1',
        hovertemplate = "<b>%{text}</b><br><br>" +
                        "Price: %{z}<br>" +
                        "<extra></extra>")) # "<extra></extra>" means we don't display the info in the secondary box, such as trace id.
    
trace1[0]['visible'] = True

trace2 = []    
    
# town_name order should be the same as "id" passed to location
for q in Types:
    trace2.append(go.Bar(
        x=Towncensus_data.sort_values([q], ascending=False).head(10)[q],
        y=Towncensus_data.sort_values([q], ascending=False).head(10)['sub_dist_1'].str.title().tolist(),
        xaxis='x2',
        yaxis='y2',
        marker=dict(
            color='rgba(91, 207, 135, 0.3)',
            line=dict(
                color='rgba(91, 207, 135, 2.0)',
                width=0.5),
        ),
        visible=False,
        name='Top 10 suburbs with the highest {} median price'.format(q),
        orientation='h',
    ))
    
trace2[0]['visible'] = True


# Maharashtra latitude and longitude values
latitude = 19.7515
longitude = 75.7139

layout = go.Layout(
    title = {'text': 'Town Census Maharashtra',
    		 'font': {'size':28, 
    		 		  'family':'Arial'}},
    autosize = True,
    
    mapbox1 = dict(
        domain = {'x': [0.3, 1],'y': [0, 1]},
        center = dict(lat=latitude, lon=longitude),
        accesstoken = mapbox_accesstoken, 
        zoom = 8),

    xaxis2={
        'zeroline': False,
        "showline": False,
        "showticklabels":True,
        'showgrid':True,
        'domain': [0, 0.25],
        'side': 'left',
        'anchor': 'x2',
    },
    yaxis2={
        'domain': [0.4, 0.9],
        'anchor': 'y2',
        'autorange': 'reversed',
    },
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(204, 204, 204)',
    plot_bgcolor='rgb(204, 204, 204)',
)
layout.update(updatemenus=list([
    dict(x=0,
         y=1,
         xanchor='left',
         yanchor='middle',
         buttons=list([
             dict(
                 args=['visible', [True, False, False, False]],
                 label='Property type:maternity',
                 method='restyle'
                 ),
             dict(
                 args=['visible', [False, True, False, False]],
                 label='Property type: maternit_1',
                 method='restyle'
                 ),
             dict(
                 args=['visible', [False, False, True, False]],
                 label='Property type: maternit_2',
                 method='restyle'
                 ),
             dict(
                 args=['visible', [False, False, False, True]],
                 label='Property type: mat_child',
                 method='restyle'
                )
            ]),
        )]))

fig=go.Figure(data=trace2 + trace1, layout=layout)
#####################################################################
# This is the part to initiate Dash app

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children=''),

    dcc.Graph(
        id='example-graph-1',
        figure=fig
    ),

    html.Div(children='''
        Data source from http://geoserver2.communitygis.net:8080/geoserver/web/
    ''')
])

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jul/2021 12:23:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 12:23:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 12:23:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 12:23:09] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2021 12:23:10] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
